<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/2019_tf/rnn-add-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.12.0


In [5]:
# let's see what compute devices we have available, hopefully a GPU 
sess = tf.Session()
devices = sess.list_devices()
for d in devices:
    print(d.name)

/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:XLA_CPU:0
/job:localhost/replica:0/task:0/device:XLA_GPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [6]:
# a small sanity check, does tf seem to work ok?
hello = tf.constant('Hello TF!')
print(sess.run(hello))

b'Hello TF!'


In [7]:
from tensorflow import keras
print(keras.__version__)

2.1.6-tf


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [10]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [11]:
questions[0]

'97+78  '

In [12]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [13]:
len(x[0])

7

In [14]:
len(questions[0])

7

In [15]:
questions[0]

'97+78  '

In [16]:
x[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [17]:
y[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [18]:
expected[0]

'175 '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [20]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


### Step 3: Create Model

In [21]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  LSTM, GRU, SimpleRNN, Dense, RepeatVector

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = LSTM
RNN = SimpleRNN # should be enough since we do not have long sequences and only local dependencies
# RNN = GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

model = Sequential()
# encoder 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# latent space
encoding_dim = 32
model.add(Dense(units=encoding_dim, activation='relu', name="encoder"))

# decoder: have 4 temporal outputs one for each of the digits of the results
model.add(RepeatVector(DIGITS + 1))

# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               18048     
_________________________________________________________________
encoder (Dense)              (None, 32)                4128      
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 32)             0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 4, 128)            20608     
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 44,332
Trainable params: 44,332
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.predict(np.array([x_val[0]]))

array([[[0.06073245, 0.08336107, 0.10029726, 0.08469585, 0.09659   ,
         0.11598717, 0.06166694, 0.06459656, 0.13300465, 0.068892  ,
         0.07243988, 0.05773613],
        [0.0650866 , 0.06419094, 0.12746704, 0.08446076, 0.10033891,
         0.13981514, 0.06703152, 0.05505698, 0.1320974 , 0.05002015,
         0.06313809, 0.05129645],
        [0.06849667, 0.06362775, 0.09853102, 0.05808814, 0.10894131,
         0.1375532 , 0.04866729, 0.05768577, 0.19622035, 0.04026084,
         0.07738415, 0.04454346],
        [0.08254389, 0.0527699 , 0.12924536, 0.06359158, 0.10203908,
         0.12213654, 0.03239036, 0.08219448, 0.17831269, 0.0339051 ,
         0.06941115, 0.05145984]]], dtype=float32)

In [24]:
model.predict_classes(np.array([x_val[0]]))

array([[8, 5, 8, 8]])

### Step 4: Train

In [25]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 6s 138us/step - loss: 1.6509 - acc: 0.3961 - val_loss: 1.4937 - val_acc: 0.4466
Q 937+13  T 950  ☒ 988 
Q 532+52  T 584  ☒ 556 
Q 818+102 T 920  ☒ 911 
Q 949+65  T 1014 ☒ 907 
Q 9+921   T 930  ☒ 991 
Q 68+468  T 536  ☒ 531 
Q 54+875  T 929  ☒ 842 
Q 87+842  T 929  ☒ 885 
Q 612+619 T 1231 ☒ 1271
Q 9+35    T 44   ☒ 56  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 6s 124us/step - loss: 1.3690 - acc: 0.4884 - val_loss: 1.2670 - val_acc: 0.5199
Q 82+499  T 581  ☒ 554 
Q 118+221 T 339  ☒ 543 
Q 719+91  T 810  ☒ 841 
Q 80+133  T 213  ☒ 122 
Q 418+910 T 1328 ☒ 1334
Q 60+386  T 446  ☒ 445 
Q 30+527  T 557  ☒ 552 
Q 341+956 T 1297 ☒ 1275
Q 39+83   T 122  ☒ 115 
Q 66+262  T 328  ☒ 320 

-------------------

In [26]:
x_val[0]

array([[False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [27]:
model.predict(np.array([x_val[0]]))

array([[[7.94129899e-13, 2.87924761e-14, 1.18646258e-25, 9.59054786e-25,
         2.03095373e-17, 7.95529961e-11, 5.67746290e-04, 9.97682333e-01,
         1.74977246e-03, 5.48088188e-08, 1.23451435e-14, 4.03143465e-22],
        [4.55670010e-17, 7.95083603e-14, 1.14772706e-18, 1.49933130e-16,
         3.47644780e-09, 9.00389254e-03, 9.90862012e-01, 1.34045709e-04,
         3.14028803e-10, 1.47472665e-13, 2.42793259e-17, 3.21213037e-20],
        [1.25700292e-11, 3.61033111e-18, 1.80906222e-19, 1.87023352e-30,
         1.95363435e-31, 2.41585786e-26, 3.99912002e-20, 1.01704859e-13,
         9.58164492e-06, 9.94372964e-01, 5.61747886e-03, 3.00486784e-12],
        [9.99999881e-01, 1.08146393e-15, 1.99482893e-17, 2.16688125e-22,
         1.51945213e-23, 1.12841766e-20, 1.43552365e-16, 5.81294561e-12,
         5.38239604e-08, 5.71181111e-08, 1.80124929e-10, 5.77742471e-14]]],
      dtype=float32)

In [28]:
model.predict_classes(np.array([x_val[0]]))

array([[7, 6, 9, 0]])